In [24]:
import os, sys, time
import tifffile as tif
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import IPython


used info in jukebox/LightSheetData/brodyatlas/atlas/original/coordinates_v1-v1.01
waxholm -> waxholm as tiff - > voxel space

waxholm z
waxholm top to bottom           = 512   0      DV
waxholm as tiff bottom to top   = 0     512    DV
    cropped 148 450
voxel x
voxel space top to bottom       = 0     302

when
xv (x in voxel) x       = 0     10     302
zwt (z in waxholm tiff) = 450   440    148
zwt = 450 - xv


waxholm y
waxholm A to P                  =   1024  0
waxholm as tiff P to A          =   0     1024
    cropped 192 948
voxel y
voxel A to P                    =   0   774

when
yv =    0   774
ywt=    948 192
ywt =   948 - yv

waxholm x
waxholm L to R                  = 512   0    ML
waxholm as tiff R to L          = 0     512
    cropped 18  473
voxel z
voxel L to R                    = 0     455

when
zv =    0  455
xwt =   473 18
xwt = 473 - zv
------------
voxel is a subspace of waxholm
to get from original waxholm:
Crop waxholm tiff at:
	xw	(ML) 	18	473
	yw	(AP)	192	948
	zw	(DV)	148 450

*and* reorient

to get bregma coordinates, need to know that
original waxholm bregma 0 0 0 is at (245,653,440)
and each voxel is 0.0390625 mm

so to get bregma coords in mm from xwt,ywt,zwt
when meas = 0.0390625 need
AP              ML              DV
(653-ywt)*meas  (245-xwt)*meas  (440-zwt)*meas

so to get bregma coords from waxholm atlas space:

AP              ML              DV
(yv -295)*meas  (zv - 228)*meas (xv - 10)*meas

In [5]:
#if you're in downloaded waxholm space (WHS_SD_rat_T2star_v1.01.nii)
# and you want to get bregma coordinates
# using info from jukebox/LightSheetData/brodyatlas/atlas/original/coordinates_v1-v1.01

#voxel coordinates in ITKsnap or FIJI coordinates
# z slices 1-512 range from ventral to dorsal (- to 0/+)
# x is ML 1-512 from animal's right (-) to left (+)
# y is 1-1024 AP from cerebellum to nose (- to +)
# original waxholm bregma 0 0 0 is at (x245,y653,z440) 
# each voxel is 0.0390625 mm

x,y,z =[400,900,340]
meas=0.0390625
AP = (y-653)*meas
ML = (x-245)*meas
DV = (z-440)*meas

print(AP,ML,DV)

9.6484375 6.0546875 -3.90625


19

In [20]:
#if you're in 'tiff space' -- 
# WHS_SD_rat_T2star_v1.01_atlas.tiff
# or WHS_SD_rat_atlas_v3_annotation.tiff
# enter your x,y,z coordinates from FIJI/ImageJ:
xt, yt, zt =[109,438,62]


# and you want to know where you are in original waxholm space

# z slices 1-455 are ML left(+) to right (-)
    # and cropped at 18 to 473 of original x1-512
# y is 1-774 AP nose to cerebellum (+ to -)
    # and cropped at  y192-948
# x is DV 1-302 dorsal to ventral (0/+ to -)
    # and cropped at z148-450

meas=0.0390625
x = 474-zt
#zt 1-455 == x473-19 because crop doesn't include 'first frame'
y = 949-yt
#yt 1-774 == y948-193
z = 450 - xt
#xt 1-302 == z450-149
print("coordinates in original waxholm space (x,y,z)")
print(x,y,z)

# or if you want to get bregma coordinates

AP = (296-yt)*meas
ML = (229-zt)*meas
DV = (10-xt)*meas
print("bregma coordinates (AP, ML, DV)")
print(AP,ML,DV)

coordinates in original waxholm space (x,y,z)
412 511 341
bregma coordinates (AP, ML, DV)
-5.546875 6.5234375 -3.8671875


In [ ]:
# to do - make a user-friendly PRA conversion for mPRA and fPRA

In [21]:
#here are your bregma coordinates!
print(AP, ML, DV)

-5.546875 6.5234375 -3.8671875


In [22]:
#here's a nice image
linktosite = "http://labs.gaidi.ca/rat-brain-atlas/?ml={}&ap={}&dv={}".format(ML,AP,DV)

In [25]:
IPython.display.IFrame(linktosite,width=800,height=800,embed=True)